In [19]:
import pandas as pd
import re
import time

t1 = time.time()

In [20]:
def change_sec(time_str):
    hmc = time_str.split(":")
    sec = int(hmc[0])*60*60 + int(hmc[1])*60 + int(hmc[2])
    return sec

def change_time(time_sec):
    sec = 0
    minu = 0
    hour = 0
    if time_sec < 60:
        sec = int(time_sec)
    elif time_sec < 3600:
        a = divmod(time_sec, 60)
        minu = int(a[0])
        sec = int(a[1])
    else:
        b = divmod(time_sec, 60*60)
        hour = int(b[0])
        if b[1] < 60:
            sec = int(b[1])
        else:
            c = divmod(b[1], 60)
            minu = int(c[0])
            sec = int(c[1])
    time = "{}:{}:{}".format(hour, minu, sec)
    return time    

In [21]:
user_tv_1 = pd.read_excel("附件1：用户收视信息.xlsx", encoding= 'gb18030', sheetname="用户收视信息")
user_tv_2 = pd.read_excel("附件1：用户收视信息.xlsx", encoding= 'gb18030', sheetname="用户回看信息")

In [22]:
user_tv_1.drop_duplicates(inplace=True)
user_tv_2.drop_duplicates(inplace=True)
user_tv_1 = user_tv_1[["设备号","频道","收看时长"]]
user_tv_2 = user_tv_2[["设备号","频道","收看时长"]]

user_tv = user_tv_1.append(user_tv_2)
user_tv.reset_index(drop=True, inplace=True)

In [23]:
for i in user_tv["频道"].values:
    if "+" in i or "-" in i:
        user_tv.loc[user_tv["频道"] == i, "频道"] = re.search(r"(.+)[+-].*", i).group(1)
user_tv.loc[user_tv["频道"] == "CCTV5", "频道"] = "中央5台"

In [24]:
user_tv["收看时长（秒）"] = [change_sec(str(i))for i in user_tv["收看时长"]]

In [25]:
channel_tag = pd.read_excel("channel_tag.xlsx", encoding="GB18030")

In [26]:
for channel in user_tv["频道"].unique():
    user_tv.loc[user_tv["频道"] == channel, "频道类型"] = channel_tag.loc[channel_tag.channel == channel, "form"].values[0]
    user_tv.loc[user_tv["频道"] == channel, "频道细分"] = channel_tag.loc[channel_tag.channel == channel, "tag"].values[0]

In [27]:
a = user_tv[["设备号", "频道", "收看时长（秒）"]].groupby(by=["设备号", "频道"], as_index=False)
grouped = a.sum().sort_values(by=["设备号", "收看时长（秒）"]).groupby("设备号", as_index=False)

user_channel_dict = {}
for i, j in grouped:
    user_channel_dict[i] = list(j.iloc[-10:, :]["频道"].values)

In [28]:
d = pd.DataFrame()
d["user"] = user_channel_dict.keys()
d["channel"] = user_channel_dict.values()

In [29]:
b = d.sort_values(by="user").reset_index().drop("index", axis=1)

In [30]:
forms = []
for user in b.values:
    channels = []
    for channel in user[1]:
        channels.append(channel_tag[channel_tag["channel"]==channel]["form"].values[0])
    forms.append(channels)

In [31]:
tags = []
for user in b.values:
    channels = []
    for channel in user[1]:
        t = channel_tag[channel_tag["channel"]==channel]["tag"].values[0]
        if type(t) is str:
            channels.append(t)
    tags.append(channels)

In [32]:
languages = []
for user in b.values:
    channels = []
    for channel in user[1]:
        channels.extend(eval(channel_tag[channel_tag["channel"]==channel]["language"].values[0]))
    languages.append(channels)

In [33]:
b["form"] = forms
b["tag"] = tags
b["language"] = languages

In [34]:
b.head()

,user,channel,form,tag,language
0,10001,"[翡翠台, 中央1台, 东方卫视]","[综合, 综合, 综合]",[],"[粤语, 国语, 国语]"
1,10002,"[中央1台, 中央少儿, 北京卫视, 中央5台, 卡酷动画, 湖南卫视, 珠江电影, 南方少...","[综合, 动画, 综合, 体育, 动画, 综合, 电影, 动画, 综合, 综合]","[动画, 各类运动, 动画, 动画]","[国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语]"
2,10003,"[快乐购物, 安徽卫视, 广州经济, 中央7台, 广东卫视, 东方卫视, 广州生活, 湖南卫...","[生活服务, 综合, 财经, 军事, 综合, 综合, 生活服务, 综合, 综合, 综合]","[购物, 生活]","[国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语]"
3,10004,"[辽宁卫视, 中央3台, 中央5台, 广东体育, 浙江卫视, 广东卫视, 北京卫视, 中央2...","[综合, 综艺, 体育, 体育, 综合, 综合, 综合, 财经, 综合, 综合]","[各类运动, 各类运动]","[国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语]"
4,10005,"[深圳卫视, 广东体育, 安徽卫视, 北京卫视, 广东卫视, 江苏卫视, 中央1台, 湖南卫...","[综合, 体育, 综合, 综合, 综合, 综合, 综合, 综合, 综合, 综合]",[各类运动],"[国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语, 国语]"


In [35]:
b.reset_index(drop=True, inplace=True)

In [36]:
b.to_csv("user_info.csv", index=False)

In [37]:
t2 = time.time()
print("用时：", t2-t1)

用时： 126.45462584495544
